# Wheat Leaves Disease Classification

Dataset credits: 
Dataset 1 : https://www.kaggle.com/datasets/olyadgetch/wheat-leaf-dataset
Dataset 2 : https://www.kaggle.com/datasets/sinadunk23/behzad-safari-jalal

### Import all the Dependencies

In [32]:
import tensorflow as tf
# from functools import partial
from tensorflow.keras import layers, models, optimizers
from genetic_algorithm import GeneticAlgorithm
import matplotlib.pyplot as plt
from IPython.display import HTML
import numpy as np
# from tensorflow.keras.metrics import Precision, Recall, Accuracy

### Import data into tensorflow dataset object

Used splitfolders tool to split dataset into training, validation and test directories.

$ pip install split-folders

$ splitfolders --ratio 0.8 0.1 0.1 -- ../../wheat_leaf


In [33]:
IMAGE_SIZE = 256
CHANNELS = 3

In [34]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        brightness_range=[0.6,1.0],
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
)
train_generator = train_datagen.flow_from_directory(
        'Balanced Dataset/train',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse",
        # save_to_dir="generated_images"
)

Found 4150 images belonging to 4 classes.


In [35]:
len(train_generator)

130

In [36]:
train_generator.class_indices

{'Brown_rust': 0, 'Healthy': 1, 'Septoria': 2, 'Yellow_rust': 3}

In [37]:
class_names = list(train_generator.class_indices.keys())
class_names

['Brown_rust', 'Healthy', 'Septoria', 'Yellow_rust']

In [38]:
count=0
for image_batch, label_batch in train_generator:
#     print(label_batch)
    print(image_batch.shape)
    break
#     count+=1
#     if count>2:
#         break

(32, 256, 256, 3)


In [39]:
validation_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True)
validation_generator = validation_datagen.flow_from_directory(
        'Balanced Dataset/val',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse"
)

Found 517 images belonging to 4 classes.


In [40]:
test_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True)

test_generator = test_datagen.flow_from_directory(
        'Balanced Dataset/test',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse"
)

Found 522 images belonging to 4 classes.


In [41]:
for image_batch, label_batch in test_generator:
    print(image_batch[0])
    break

[[[0.23889874 0.27419287 0.08235294]
  [0.25490198 0.2901961  0.08235294]
  [0.25490198 0.2901961  0.08235294]
  ...
  [0.97573537 0.8995788  0.74360454]
  [0.96779794 0.8923855  0.73541903]
  [0.95986044 0.8851922  0.7272336 ]]

 [[0.23741047 0.27270457 0.08235294]
  [0.25490198 0.2901961  0.08235294]
  [0.25490198 0.2901961  0.08235294]
  ...
  [0.84824514 0.78382975 0.6122006 ]
  [0.83956355 0.7758923  0.603271  ]
  [0.8308821  0.7679548  0.59434134]]

 [[0.2359222  0.2712163  0.08235294]
  [0.25490198 0.2901961  0.08235294]
  [0.25490198 0.2901961  0.08235294]
  ...
  [0.72537386 0.66304624 0.48690075]
  [0.72140515 0.656597   0.48318005]
  [0.71743643 0.6501479  0.4794594 ]]

 ...

 [[0.42209026 0.34510314 0.02580113]
  [0.4168813  0.34039026 0.02456091]
  [0.41167238 0.33567744 0.02332068]
  ...
  [0.5882353  0.4784314  0.3254902 ]
  [0.5882353  0.4784314  0.3254902 ]
  [0.58507204 0.4815947  0.3476333 ]]

 [[0.5164561  0.4232209  0.12126009]
  [0.52761817 0.4326466  0.1306858 ]


## Building the Model

In [55]:
import random
from deap import base, creator, tools
import itertools

input_shape = (256, 256, 3)
n_classes = 4

possible_activations = ['relu', 'sigmoid', 'tanh']
possible_optimizers = ['adam', 'sgd', 'rmsprop']
possible_pooling = ['max', 'average']
possible_learning_rates = [0.001, 0.01, 0.1]

possible_values = [possible_activations, possible_optimizers, possible_pooling, possible_learning_rates]
possible_values = [list(x) if isinstance(x, tuple) else x for x in possible_values]
# set genetic algorithm parameters
population_size = 20
mutation_probability = 0.05
crossover_probability = 0.9
num_generations = 10
tournament_size = 3

# create the fitness function
def fitness_function(individual):
    activation = individual[0]
    optimizer = individual[1]
    pooling = individual[2]
    learning_rate = individual[3]
    print("Hyperparameters :", individual)
    input_shape = (256, 256, 3)
    n_classes = 4
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation=activation),
        tf.keras.layers.Dense(n_classes, activation='softmax'),
    ])
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=['accuracy']
    )
    history = model.fit(
        train_generator,
        steps_per_epoch=129,
        batch_size=32,
        validation_data=validation_generator,
        validation_steps=16,
        verbose=0,
        epochs=5,
    )
    fitness = history.history["accuracy"][-1]
    print("accuracy :", fitness)
    return fitness,

# create the toolbox
toolbox = base.Toolbox()

# create a function to generate a random gene
def random_gene():
    return random.randint(0, len(possible_activations)-1)

# create a function to generate a chromosome
def create_chromosome(num_genes):
    return [random_gene() for _ in range(num_genes)]

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_activation", random.choice, possible_activations)
toolbox.register("attr_optimizer", random.choice, possible_optimizers)
toolbox.register("attr_pooling", random.choice, possible_pooling)
toolbox.register("attr_learning_rate", random.choice, possible_learning_rates)
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_activation, toolbox.attr_optimizer,
                  toolbox.attr_pooling, toolbox.attr_learning_rate), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_function)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(possible_values)-1, indpb=mutation_probability)
toolbox.register("select", tools.selTournament, tournsize=3)

def optimize_hyperparameters():
    # create initial population
    population = toolbox.population(n=20)
    # evaluate initial fitness
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = (fit,)
    # start evolution
    for g in range(num_generations):
        print(f"Generation {g+1}")
        # select parents
        parents = toolbox.select(population, k=n_parents)
        # clone parents
        offspring = list(map(toolbox.clone, parents))
        # apply crossover and mutation
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < crossover_prob:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        for mutant in offspring:
            if random.random() < mutation_prob:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        # evaluate fitness of new individuals
        fitnesses = list(map(toolbox.evaluate, offspring))
        for ind, fit in zip(offspring, fitnesses):
            ind.fitness.values = (fit,)
        # select new population
        population = toolbox.select(population + offspring, k=population_size)
    best_ind = tools.selBest(population, 1)[0]
    return best_ind.fitness.values, best_ind


best_fitness, best_ind = optimize_hyperparameters()
print("Best fitness:", best_fitness)
print("Best individual:", best_ind)


Hyperparameters : ['sigmoid', 'adam', 'average', 0.001]
accuracy : 0.28411850333213806
Hyperparameters : ['tanh', 'sgd', 'average', 0.1]
accuracy : 0.7476930618286133
Hyperparameters : ['sigmoid', 'sgd', 'average', 0.01]
accuracy : 0.2812044620513916
Hyperparameters : ['tanh', 'adam', 'max', 0.001]
accuracy : 0.8878096342086792
Hyperparameters : ['tanh', 'rmsprop', 'max', 0.1]
accuracy : 0.8598834276199341
Hyperparameters : ['relu', 'rmsprop', 'average', 0.1]
accuracy : 0.8455560803413391
Hyperparameters : ['tanh', 'sgd', 'average', 0.001]
accuracy : 0.7447789907455444
Hyperparameters : ['sigmoid', 'adam', 'max', 0.001]
accuracy : 0.2795046269893646
Hyperparameters : ['relu', 'rmsprop', 'max', 0.1]
accuracy : 0.8725109100341797
Hyperparameters : ['sigmoid', 'adam', 'average', 0.01]
accuracy : 0.2880038917064667
Hyperparameters : ['sigmoid', 'rmsprop', 'average', 0.001]
accuracy : 0.282661497592926
Hyperparameters : ['sigmoid', 'sgd', 'max', 0.1]
accuracy : 0.2850898504257202
Hyperparam

TypeError: Both weights and assigned values must be a sequence of numbers when assigning to values of <class 'deap.creator.FitnessMax'>. Currently assigning value(s) ((0.28411850333213806,),) of <class 'tuple'> to a fitness with weights (1.0,).

In [56]:
    activation = 'relu'
    pooling = 'max'
    learning_rate = 0.01
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    input_shape = (256, 256, 3)
    n_classes = 4
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation=activation),
        tf.keras.layers.Dense(n_classes, activation='softmax'),
    ])
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=['accuracy']
    )
    history = model.fit(
        train_generator,
        steps_per_epoch=129,
        batch_size=32,
        validation_data=validation_generator,
        validation_steps=16,
        verbose=0,
        epochs=5,
    )
    fitness = history.history["accuracy"][-1]
    print("accuracy :", fitness)

accuracy : 0.28897523880004883


In [57]:
    activation = 'relu'
    pooling = 'max'
    learning_rate = 0.1
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    input_shape = (256, 256, 3)
    n_classes = 4
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation=activation),
        getattr(tf.keras.layers, f"{pooling.capitalize()}Pooling2D")((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation=activation),
        tf.keras.layers.Dense(n_classes, activation='softmax'),
    ])
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=['accuracy']
    )
    history = model.fit(
        train_generator,
        steps_per_epoch=129,
        batch_size=32,
        validation_data=validation_generator,
        validation_steps=16,
        verbose=0,
        epochs=5,
    )
    fitness = history.history["accuracy"][-1]
    print("accuracy :", fitness)

accuracy : 0.2768334150314331
